In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge

# 1. Cargar dataset original
df = pd.read_csv("sell-in.txt", sep="\t")
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")

# 2. Pivotear datos para obtener series por producto
df_pivot = df.groupby(["periodo", "product_id"])["tn"].sum().unstack().sort_index()

# 3. Cargar listado de productos a predecir (780)
productos_objetivo = pd.read_csv("product_id_apredecir201912.txt")["product_id"].tolist()

# 4. Generar lista de productos con al menos 12 meses antes de febrero 2020
ultimo_mes = pd.Timestamp("2019-12-01")
productos_validos = []

for producto in productos_objetivo:
    if producto not in df_pivot.columns:
        continue
    serie = df_pivot[producto].loc[:ultimo_mes]
    if serie.notna().sum() >= 12:
        productos_validos.append(producto)

# 5. Inicializar resultados
resultados = []

# 6. Entrenar un modelo por producto
for producto in productos_validos:
    serie = df_pivot[producto].loc[:ultimo_mes].dropna()
    if len(serie) < 12:
        continue

    X = []
    y = []
    for i in range(len(serie) - 12):
        X.append(serie.iloc[i:i+12].values)
        y.append(serie.iloc[i+12])

    if not X:
        continue

    X = np.array(X)
    y = np.array(y)

    model = Ridge()
    model.fit(X, y)

    # Predicción para febrero 2020
    ultima_ventana = serie.iloc[-12:].values.reshape(1, -1)
    pred = model.predict(ultima_ventana)[0]

    resultados.append((producto, pred))

# 7. Fallback: promedio últimos 3 meses
productos_faltantes = set(productos_objetivo) - set(productos_validos)
for producto in productos_faltantes:
    if producto not in df_pivot.columns:
        resultados.append((producto, 0.0))
        continue
    serie = df_pivot[producto].dropna().loc[:ultimo_mes]
    if len(serie) >= 3:
        pred = serie.iloc[-3:].mean()
    else:
        pred = 0.0
    resultados.append((producto, pred))

# 8. Guardar resultados (sólo 780 productos)
df_resultados = pd.DataFrame(resultados, columns=["product_id", "tn_predicho"])
df_resultados = df_resultados[df_resultados["product_id"].isin(productos_objetivo)]
df_resultados = df_resultados.sort_values("product_id")

# Validación final
assert df_resultados.shape[0] == 780, "❌ El archivo de salida no tiene 780 filas."

df_resultados.to_csv("predicciones_febrero2020_regresion_simple.csv", index=False)
print("✅ Archivo guardado: predicciones_febrero2020_regresion_simple.csv")


✅ Archivo guardado: predicciones_febrero2020_regresion_simple.csv
